# DeepLabCut Implimentation 

Here our goal is to create marker predict using Deeplabcut. The output of this code will be the Marker predictions file(.csv file) and a video with the marker predictions on the animal. 

In [ ]:
#mount your google drive
from google.colab import drive
drive.mount('/content/drive')

To obtain the marker predictions, we need the config file created in the previous step. The directories below should be modified to include the locations of these files on your local machine

In [ ]:
#Change your current directory so you are in the folder where you config file is located 
%cd /content/drive/My Drive/trial3-team12-2019-09-19

Install deeplabcut so that you can use the build in functions created by DeepLabCut to obtain the marker predictions. 

In [ ]:
#install deeplabcut 
!pip install deeplabcut 

Install an tensorflow versoun 1.13 becuase DeepLabCut only runs on an older version of tensor flow. If you do not do this step, DeepLabCut will not run. 

In [ ]:
#install tensor flow - you need to do this step becuase DeepLabCut runs on an older version of tensor flow 
!pip install tensorflow==1.13.1

Now, import all your librabries. 

In [ ]:
# library imports
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

import tensorflow as tf
import deeplabcut
import pandas as pd
from pandas_datareader import data, wb

The directories below should be modified to include the locations of the config file on your google drive. This step sets the variable you are going to pass in when you train the videos. 

In [ ]:
#change your path file to the correct directory 
path_config_file = '/content/drive/My Drive/trial4/2ratsIR_trial-team12-2019-10-11/config_google.yaml' #this just sets the variable that your gonna pass in when ur gonna train

This step allows you to create the training dataset. Set the config file below to the video you would like to train. 

In [ ]:
#create training dataset
deeplabcut.create_training_dataset(path_config_file,num_shuffles=1)

Now you can train the network. It is recommended to train for thousands of iterations until the loss plateaus (typically >200,000). The variables ‘display_iters’ and ‘save_iters’ in the pose_cfg.yaml file allows the user to alter how often the loss is displayed and how often the weights are stored.The function ‘train_network’ helps the user in training the network. 

In [ ]:
#train the network
deeplabcut.train_network(path_config_file,maxiters=10000,displayiters=100)

It is important to evaluate the performance of the trained network. This performance is measured by computing the mean average Euclidean error (MAE; which is proportional to the average root mean square error) between the manual labels and the ones predicted by DeepLabCut. The MAE is saved as a comma separated file and displayed for all pairs and only likely pairs (>p-cutoff). This helps to exclude, for example, occluded body parts. One of the strengths of DeepLabCut is that due to the probabilistic output of the scoremap, it can, if sufficiently trained, also reliably report if a body part is visible in a given frame. Setting ‘plotting’ to true plots all the testing and training frames with the manual and predicted labels. The user should visually check the labeled test (and training) images that are created in the ‘evaluation-results’ directory. Ideally, DeepLabCut labeled unseen (test images) according to the user’s required accuracy, and the average train and test errors are comparable (good generalization). What (numerically) comprises an acceptable MAE depends on many factors (including the size of the tracked body parts, the labeling variability, etc.). Note that the test error can also be larger than the training error due to human variability. The plots can be customized by editing the config.yaml file (i.e. the colormap, scale, marker size (dotsize), and transparency of labels (alphavalue) can be modified). By default each body part is plotted in a different color (governed by the colormap) and the plot labels indicate their source. Note that by default the human labels are plotted as plus (‘+’), DeepLabCut’s predictions either as ‘.’ (for confident predictions with likelihood > p-cutoff) and ’x’ for (likelihood <= p-cutoff). The evaluation results for each shuffle of the training dataset are stored in a unique subdirectory in a newly created directory ‘evaluation-results’ in the project directory. The user can visually inspect if the distance between the labeled and the predicted body parts is acceptable. In the event of benchmarking with different shuffles of same training dataset, the user can provide multiple shuffle indices to evaluate the corresponding network


In [ ]:
#evalulate the network 
deeplabcut.evaluate_network(path_config_file,plotting=True)

The trained network can be used to analyze new videos. The user needs to first choose a checkpoint with the best evaluation results for analyzing the videos. In this case, the user can enter the corresponding index of the checkpoint to the variable snapshotindex in the config.yaml file. By default, the most recent checkpoint (i.e. last) is used for analyzing the video.

In [ ]:
#analyze the video 
deeplabcut.analyze_videos(path_config_file,['/content/drive/My Drive/trial4/2ratsIR_trial-team12-2019-10-11/videos/2ratsIRbag.avi'],save_as_csv=True,videotype='.avi')

The labels are stored in a MultiIndex Pandas Array ( http://pandas.pydata.org), which contains the name of the network, body part name, (x, y) label position in pixels, and the likelihood for each frame per body part. These arrays are stored in an efficient Hierarchical Data Format (HDF) in the same directory, where the video is stored. However, if the flag save_as_csv is set to True, the data can also be exported in comma-separated values format (.csv), which in turn can be imported in many programs, such as MATLAB, R, Prism, etc.; This flag is set to False by default. The labels for each body part across the video (‘trajectories’) can also be plotted after analyze_videos is run. The provided plotting function in this toolbox utilizes matplotlib therefore these plots can easily be customized by the end user. 

In [ ]:
#create labeled video 
deeplabcut.create_labeled_video(path_config_file,['/content/drive/My Drive/trial4/2ratsIR_trial-team12-2019-10-11/videos/2ratsIRbag.avi'])

In [ ]:
#plot trajectories 
deeplabcut.plot_trajectories(config_path,['/content/drive/My Drive/trial4/2ratsIR_trial-team12-2019-10-11/videos/2ratsIRbag.avi'])
